# Processing words as a sequence: The sequence model approach

## download the data

```sh
pdm run data_imdb
```

In [1]:
import os
import pathlib
import random
import shutil

In [2]:
base_dir = pathlib.Path("./data/acl_imdb/aclImdb")


def prepare_val_data():
    val_dir = base_dir / "val"
    if val_dir.exists():
        print("validation data already existed")
        return
    train_dir = base_dir / "train"
    for category in ("neg", "pos"):
        os.makedirs(val_dir / category)
        files = os.listdir(train_dir / category)
        random.Random(1337).shuffle(files)
        num_val_samples = int(0.2 * len(files))
        val_files = files[-num_val_samples:]
        for fname in val_files:
            shutil.move(train_dir / category / fname, val_dir / category / fname)


prepare_val_data()

validation data already existed


In [3]:
from tensorflow import keras

batch_size = 32

train_ds = keras.utils.text_dataset_from_directory(
    f"{base_dir}/train", batch_size=batch_size
)
val_ds = keras.utils.text_dataset_from_directory(
    f"{base_dir}/val", batch_size=batch_size
)
test_ds = keras.utils.text_dataset_from_directory(
    f"{base_dir}/test", batch_size=batch_size
)
text_only_train_ds = train_ds.map(lambda x, y: x)

Found 16000 files belonging to 2 classes.
Metal device set to: Apple M1 Max


2023-02-11 08:44:03.821551: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-02-11 08:44:03.821972: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Found 4000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.


In [4]:
from tensorflow.keras import layers

CPU_CORES = 8

max_length = 600
max_tokens = 20000
text_vectorization = layers.TextVectorization(
    max_tokens=max_tokens,
    output_mode="int",
    # make the input size manageable, truncate after the first 600 words
    output_sequence_length=max_length,
)
text_vectorization.adapt(text_only_train_ds)

int_train_ds = train_ds.map(
    lambda x, y: (text_vectorization(x), y), num_parallel_calls=CPU_CORES
)
int_val_ds = val_ds.map(
    lambda x, y: (text_vectorization(x), y), num_parallel_calls=CPU_CORES
)
int_test_ds = test_ds.map(
    lambda x, y: (text_vectorization(x), y), num_parallel_calls=CPU_CORES
)

2023-02-11 08:45:30.852208: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-02-11 08:45:30.892764: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


## A sequence model built on one-hot encoded vector sequences

In [5]:
import tensorflow as tf

inputs = keras.Input(shape=(None,), dtype="int64")
embedded = tf.one_hot(inputs, depth=max_tokens)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop", loss="binary_crossentropy", metrics=["accuracy"])
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None)]            0         
                                                                 
 tf.one_hot (TFOpLambda)     (None, None, 20000)       0         
                                                                 
 bidirectional (Bidirectiona  (None, 64)               5128448   
 l)                                                              
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense (Dense)               (None, 1)                 65        
                                                                 
Total params: 5,128,513
Trainable params: 5,128,513
Non-trainable params: 0
___________________________________________________

In [12]:
callbacks = [
    keras.callbacks.ModelCheckpoint("one_hot_bidir_lstm.keras", save_best_only=True)
]
model.fit(int_train_ds, validation_data=int_val_ds, epochs=10, callbacks=callbacks)
model = keras.models.load_model("one_hot_bidir_lstm.keras")
print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

Epoch 1/10


2023-02-11 08:48:20.293696: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-02-11 08:48:20.571841: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-02-11 08:48:20.711348: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-02-11 08:48:23.257977: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-02-11 08:48:23.276232: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


500/500 [==============================] - ETA: 0s - loss: 0.5543 - accuracy: 0.7224

2023-02-11 08:51:35.532203: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-02-11 08:51:35.604846: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-02-11 08:51:35.615220: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


500/500 [==============================] - 229s 442ms/step - loss: 0.5543 - accuracy: 0.7224 - val_loss: 0.3761 - val_accuracy: 0.8408
Epoch 2/10
500/500 [==============================] - 218s 435ms/step - loss: 0.3429 - accuracy: 0.8722 - val_loss: 0.6043 - val_accuracy: 0.8233
Epoch 3/10
304/500 [=================>............] - ETA: 1:13 - loss: 0.2712 - accuracy: 0.9071

KeyboardInterrupt: 

## Learning word embeddings with the Embedding layer

In [13]:
embedding_layer = layers.Embedding(input_dim=max_tokens, output_dim=256)

In [14]:
inputs = keras.Input(shape=(None,), dtype="int64")
embedded = layers.Embedding(input_dim=max_tokens, output_dim=256)(inputs)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop", loss="binary_crossentropy", metrics=["accuracy"])
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding_1 (Embedding)     (None, None, 256)         5120000   
                                                                 
 bidirectional_1 (Bidirectio  (None, 64)               73984     
 nal)                                                            
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 5,194,049
Trainable params: 5,194,049
Non-trainable params: 0
_________________________________________________

In [15]:
callbacks = [
    keras.callbacks.ModelCheckpoint("embeddings_bidir_gru.keras", save_best_only=True)
]
model.fit(int_train_ds, validation_data=int_val_ds, epochs=10, callbacks=callbacks)
model = keras.models.load_model("embeddings_bidir_gru.keras")
print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

Epoch 1/10


2023-02-11 09:00:37.879515: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-02-11 09:00:38.102896: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-02-11 09:00:38.115811: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-02-11 09:00:39.659393: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-02-11 09:00:39.677406: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


500/500 [==============================] - ETA: 0s - loss: 0.4855 - accuracy: 0.7747

2023-02-11 09:03:11.880390: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-02-11 09:03:11.953214: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-02-11 09:03:11.963059: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


500/500 [==============================] - 174s 336ms/step - loss: 0.4855 - accuracy: 0.7747 - val_loss: 0.4362 - val_accuracy: 0.8003
Epoch 2/10
500/500 [==============================] - 163s 326ms/step - loss: 0.3005 - accuracy: 0.8902 - val_loss: 0.3654 - val_accuracy: 0.8638
Epoch 3/10
500/500 [==============================] - 161s 323ms/step - loss: 0.2334 - accuracy: 0.9185 - val_loss: 0.3503 - val_accuracy: 0.8668
Epoch 4/10
500/500 [==============================] - 163s 326ms/step - loss: 0.1931 - accuracy: 0.9350 - val_loss: 0.3865 - val_accuracy: 0.8693
Epoch 5/10
500/500 [==============================] - 166s 331ms/step - loss: 0.1561 - accuracy: 0.9462 - val_loss: 0.3829 - val_accuracy: 0.8780
Epoch 6/10
500/500 [==============================] - 167s 335ms/step - loss: 0.1280 - accuracy: 0.9550 - val_loss: 0.4642 - val_accuracy: 0.8460
Epoch 7/10
500/500 [==============================] - 163s 326ms/step - loss: 0.1058 - accuracy: 0.9660 - val_loss: 0.4332 - val_accura

2023-02-11 09:28:11.140483: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-02-11 09:28:11.220607: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-02-11 09:28:11.232083: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


782/782 [==============================] - 118s 148ms/step - loss: 0.3829 - accuracy: 0.8505
Test acc: 0.851


## Using an Embedding layer with masking enabled

In [16]:
inputs = keras.Input(shape=(None,), dtype="int64")
embedded = layers.Embedding(input_dim=max_tokens, output_dim=256, mask_zero=True)(
    inputs
)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop", loss="binary_crossentropy", metrics=["accuracy"])
model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding_2 (Embedding)     (None, None, 256)         5120000   
                                                                 
 bidirectional_2 (Bidirectio  (None, 64)               73984     
 nal)                                                            
                                                                 
 dropout_2 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 1)                 65        
                                                                 
Total params: 5,194,049
Trainable params: 5,194,049
Non-trainable params: 0
_________________________________________________

In [17]:
callbacks = [
    keras.callbacks.ModelCheckpoint(
        "embeddings_bidir_gru_with_masking.keras", save_best_only=True
    )
]
model.fit(int_train_ds, validation_data=int_val_ds, epochs=5, callbacks=callbacks)
model = keras.models.load_model("embeddings_bidir_gru_with_masking.keras")
print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

Epoch 1/5


2023-02-11 09:30:11.006655: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-02-11 09:30:11.448265: W tensorflow/core/common_runtime/forward_type_inference.cc:332] Type inference failed. This indicates an invalid graph that escaped type checking. Error message: INVALID_ARGUMENT: expected compatible input types, but input 1:
type_id: TFT_OPTIONAL
args {
  type_id: TFT_PRODUCT
  args {
    type_id: TFT_TENSOR
    args {
      type_id: TFT_INT32
    }
  }
}
 is neither a subtype nor a supertype of the combined inputs preceding it:
type_id: TFT_OPTIONAL
args {
  type_id: TFT_PRODUCT
  args {
    type_id: TFT_TENSOR
    args {
      type_id: TFT_FLOAT
    }
  }
}

	while inferring type of node 'cond_40/output/_23'
2023-02-11 09:30:11.452266: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-02-11 09:30:11.594800: I tensorflow/co

500/500 [==============================] - ETA: 0s - loss: 0.6944 - accuracy: 0.4991

2023-02-11 09:33:01.571281: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-02-11 09:33:01.775600: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-02-11 09:33:01.850049: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


500/500 [==============================] - 194s 371ms/step - loss: 0.6944 - accuracy: 0.4991 - val_loss: 0.6930 - val_accuracy: 0.5058
Epoch 2/5
500/500 [==============================] - 180s 360ms/step - loss: 0.6913 - accuracy: 0.5096 - val_loss: 0.6934 - val_accuracy: 0.5078
Epoch 3/5
500/500 [==============================] - 179s 358ms/step - loss: 0.6823 - accuracy: 0.5122 - val_loss: 0.6946 - val_accuracy: 0.5100
Epoch 4/5
500/500 [==============================] - 175s 351ms/step - loss: 0.6680 - accuracy: 0.5213 - val_loss: 0.6988 - val_accuracy: 0.5143
Epoch 5/5
500/500 [==============================] - 175s 349ms/step - loss: 0.6624 - accuracy: 0.5269 - val_loss: 0.6974 - val_accuracy: 0.5130


2023-02-11 09:45:13.206980: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-02-11 09:45:13.401319: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-02-11 09:45:13.494236: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


782/782 [==============================] - 130s 163ms/step - loss: 0.6932 - accuracy: 0.5027
Test acc: 0.503


## Using pretrained word embeddings

In [19]:
import numpy as np

path_to_glove_file = "./data/glove/glove.6B.100d.txt"

embeddings_index = {}
with open(path_to_glove_file) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print(f"Found {len(embeddings_index)} word vectors.")

Found 400000 word vectors.


### Preparing the GloVe word-embeddings matrix

In [20]:
embedding_dim = 100

vocabulary = text_vectorization.get_vocabulary()
word_index = dict(zip(vocabulary, range(len(vocabulary))))

embedding_matrix = np.zeros((max_tokens, embedding_dim))
for word, i in word_index.items():
    if i < max_tokens:
        embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [21]:
embedding_layer = layers.Embedding(
    max_tokens,
    embedding_dim,
    embeddings_initializer=keras.initializers.Constant(embedding_matrix),
    trainable=False,
    mask_zero=True,
)

In [22]:
inputs = keras.Input(shape=(None,), dtype="int64")
embedded = embedding_layer(inputs)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop", loss="binary_crossentropy", metrics=["accuracy"])
model.summary()

callbacks = [
    keras.callbacks.ModelCheckpoint(
        "glove_embeddings_sequence_model.keras", save_best_only=True
    )
]
model.fit(int_train_ds, validation_data=int_val_ds, epochs=5, callbacks=callbacks)
model = keras.models.load_model("glove_embeddings_sequence_model.keras")
print(f"Test acc: {model.evaluate(int_test_ds)[1]:.3f}")

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding_3 (Embedding)     (None, None, 100)         2000000   
                                                                 
 bidirectional_3 (Bidirectio  (None, 64)               34048     
 nal)                                                            
                                                                 
 dropout_3 (Dropout)         (None, 64)                0         
                                                                 
 dense_3 (Dense)             (None, 1)                 65        
                                                                 
Total params: 2,034,113
Trainable params: 34,113
Non-trainable params: 2,000,000
____________________________________________

2023-02-11 09:49:18.395251: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-02-11 09:49:18.856860: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-02-11 09:49:19.012475: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-02-11 09:49:20.976055: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-02-11 09:49:21.106416: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


500/500 [==============================] - ETA: 0s - loss: 0.6939 - accuracy: 0.5021

2023-02-11 09:51:36.595330: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-02-11 09:51:36.802027: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-02-11 09:51:36.878576: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


500/500 [==============================] - 164s 310ms/step - loss: 0.6939 - accuracy: 0.5021 - val_loss: 0.6940 - val_accuracy: 0.5068
Epoch 2/5
500/500 [==============================] - 148s 295ms/step - loss: 0.6915 - accuracy: 0.5039 - val_loss: 0.6917 - val_accuracy: 0.5090
Epoch 3/5
500/500 [==============================] - 148s 296ms/step - loss: 0.6897 - accuracy: 0.5038 - val_loss: 0.6925 - val_accuracy: 0.5090
Epoch 4/5
500/500 [==============================] - 148s 297ms/step - loss: 0.6879 - accuracy: 0.5166 - val_loss: 0.6891 - val_accuracy: 0.5153
Epoch 5/5
500/500 [==============================] - 148s 297ms/step - loss: 0.6862 - accuracy: 0.5109 - val_loss: 0.6883 - val_accuracy: 0.5145


2023-02-11 10:01:53.622187: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-02-11 10:01:53.821369: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-02-11 10:01:53.907254: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


782/782 [==============================] - 131s 164ms/step - loss: 0.6891 - accuracy: 0.5118
Test acc: 0.512
